In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys

import torch
from torchvision.io import read_image
from torchvision import models, transforms, datasets

def close_window():
    cv2.destroyAllWindows()  # Close the window
    cv2.waitKey(1)
    cv2.waitKey(1)
    
def ifNotExistExit(filename):
    if not filename:
        print('filename is None')
        sys.exit()

## Webcam 동영상 열기

In [3]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Camera open failed!')
    sys.exit()

# Print current camera properties
print(f"Frame width: {cap.get(cv2.CAP_PROP_FRAME_WIDTH)}")
print(f"Frame height: {cap.get(cv2.CAP_PROP_FRAME_HEIGHT)}")

# Update camera properties
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920 // 3)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080 // 2.25)

# Print updated camera properties
print('Frame width:', int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
print('Frame height:', int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

# Process camera frames
while True:
    ret, frame = cap.read()

    if not ret:
        break

    edge = cv2.Canny(frame, 50, 150)
    
    cv2.imshow('frame', frame)
    cv2.imshow('frame1', edge)

    if cv2.waitKey(10) == 27: # ESC key to exit
        break

# cap.release()
# cv2.destroyAllWindows()

cap.release()
cv2.waitKey() # wait until any key,'q' is pressed
close_window()


Frame width: 1280.0
Frame height: 720.0
Frame width: 640
Frame height: 480


2025-04-07 20:02:02.426 python[88489:2402866] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-07 20:02:02.426 python[88489:2402866] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## OpenCV DNN webcam 얼굴검출
- https://github.com/spmallick/learnopencv/tree/master/AgeGender

In [4]:
import sys
import numpy as np
import cv2

## Tensorflow 학습모델
model = './opencv_face_detector_uint8.pb'
config = './opencv_face_detector.pbtxt'
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Camera open failed!')
    sys.exit()

net = cv2.dnn.readNet(model, config)
if net.empty():
    print('Net open failed!')
    sys.exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123))
    net.setInput(blob)
    out = net.forward()# out.shape=(1,1, 200, 7)
    detect = out[0, 0, :, :] ##0, 0, 사용안함
    (h, w) = frame.shape[:2]

    for i in range(detect.shape[0]):
        confidence = detect[i, 2]
        if confidence > 0.5:
            # out matrix에서 x1, y1, x2, y2 값이 0 ~1로 normalize 되어 있음
            x1 = int(detect[i, 3] * w)
            y1 = int(detect[i, 4] * h)
            x2 = int(detect[i, 5] * w)
            y2 = int(detect[i, 6] * h)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0))
            label = f'Face: {confidence:4.2f}'
            cv2.putText(frame, label, (x1, y1 - 1), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.8, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == 27:
        break

# cap.release()
# cv2.destroyAllWindows()
cv2.waitKey() # wait until any key,'q' is pressed
close_window()

2025-04-07 21:06:54.347 python[88489:2402866] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
2025-04-07 21:06:54.347 python[88489:2402866] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
